In [44]:
import scipy
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
#from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torchvision.models as models
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
#%run Preprocessing_SVHN_data.ipynb
%matplotlib inline
from fastai.vision.widgets import *
import time

In [2]:
#! pip install mlcrate

In [3]:
from fastai.vision.all import *
from fastai.metrics import error_rate
import mlcrate as mlc
import matplotlib.pyplot as plt
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [38]:
#print(torch.cuda.current_device())

In [36]:
!nvidia-smi

Mon Nov  2 13:57:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [57]:
transforms_final = transforms.Compose([ transforms.Resize(32),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomRotation(30),
                                        #transforms.Grayscale(),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.4516),(0.1993))])

In [58]:
train_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/train", split = "train", download = False, transform = transforms_final, target_transform = None)
validation_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/test", split = "test", download = False, transform = transforms_final, target_transform = None)
extra_data = torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/extra", split = "extra", download = False, transform = transforms_final, target_transform = None)

In [6]:
#train_set = torch.utils.data.ConcatDataset([train_set, extra_set])

In [59]:
batch_size = 64

In [60]:
train_loader = DataLoader(train_data, batch_size, shuffle=True, pin_memory = True, num_workers = 4)
val_loader = DataLoader(validation_data, batch_size, pin_memory = True, num_workers = 4)

In [61]:
#wide_resnet50_2 = models.wide_resnet50_2(pretrained = True)

In [62]:
#wide_resnet50_2

In [63]:
# time = mlc.time.Timer()
# learn = Learner.fit_one_cycle(train_loader,models.wide_resnet50_2(pretrained = True), metrics = [error_rate, accuracy])
# print("Train Cycle 1")
# learn.fit_one_cycle(8)
# learn.recorder.plot_losses() 
# print("Time taken: {}".format(time.since(0)))

In [64]:
image_datasets = {x: torchvision.datasets.SVHN("/home/jupyter/pytorch_datasets/" + x, split = x, download = False, transform = transforms_final, target_transform = None)
                  for x in ['train', 'test']}

In [65]:
image_datasets

{'train': Dataset SVHN
     Number of datapoints: 73257
     Root location: /home/jupyter/pytorch_datasets/train
     Split: train
     StandardTransform
 Transform: Compose(
                Resize(size=32, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                RandomRotation(degrees=(-30, 30), resample=False, expand=False)
                ToTensor()
                Normalize(mean=0.4516, std=0.1993)
            ),
 'test': Dataset SVHN
     Number of datapoints: 26032
     Root location: /home/jupyter/pytorch_datasets/test
     Split: test
     StandardTransform
 Transform: Compose(
                Resize(size=32, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                RandomRotation(degrees=(-30, 30), resample=False, expand=False)
                ToTensor()
                Normalize(mean=0.4516, std=0.1993)
            )}

In [66]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}

In [67]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f8bf49c4150>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f8bf49c2210>}

In [68]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
dataset_sizes

{'train': 73257, 'test': 26032}

In [69]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [70]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [71]:
model_ft = models.wide_resnet50_2(pretrained = True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [72]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [74]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=5)

Epoch 0/4
----------
train Loss: 1.0237 Acc: 0.6303


KeyError: 'val'

In [ ]:
visualize_model(model_ft)

In [ ]:
# model_ft = models.wide_resnet50_2(pretrained = True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 10)

# model_conv = model_conv.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
# model_conv = train_model(model_conv, criterion, optimizer_conv,exp_lr_scheduler, num_epochs=25)

In [ ]:
# visualize_model(model_conv)

# plt.ioff()
# plt.show()